[![Kaggle link](https://img.shields.io/badge/изначальный%20датасет-Открыть-orange?style=social&logo=kaggle)](https://www.kaggle.com/datasets/leomartinelli/bullying-in-schools)


# 1. Устранение пропусков в данных

In [129]:
import pandas as pd
from tabulate import tabulate
import numpy as np
%matplotlib inline 

In [130]:
data.shape

(56981, 18)

In [131]:
df_csv_path = 'data/Bullying_2018_RAW.csv'
data = pd.read_csv(df_csv_path) 

In [132]:
print(tabulate([data.shape],headers=('Строки', 'Столбцы'),tablefmt="github"))

|   Строки |   Столбцы |
|----------|-----------|
|    56981 |        18 |


## Описание датасета

In [133]:

headers=('Номер','Название','Описание','Кат.','Числ.','Кол-во вариаций')

column_desc = [
    ['Номер ответа','-'],
    ['В школе (за последние 12 месяцев)','Cat.'], # - №2
    ['Вне школы (за последние 12 месяцев)','Cat.'], # - №3
    ['Кибербуллинг (последние 12 месяцев?)','Cat.'], # - №3
    ['Возраст респондента','Num.'], # - №4
    ['Пол респондента','Cat.'], # - №5
    ['Количество физических атак','Num.'], # - №6
    ['Количество драк','Num.'], # # - №7
    ['Ощущение одинокости','Cat.'], # # - №8
    ['Количество близких друзей','Num.'], # - №9
    ['Количество дней пропусков школы без разрешения','Num'], # - №10
    ['Доброта и отзывчивость окружающих','Cat.'], # - №11
    ['Частота проблем взаимопонимания с родителями','Cat.'], # - №12
    ['Ощущение одинокости большую части (частота)','Cat.'], # # - №13
    ['Пропускали ли вы школу без разрешения (да/нет)','Cat.'], # - №14
    ['Масса вашего тела в школе была ниже нормы?','Cat.'], # - №15
    ['Масса вашего тела в школе была выше нормы?','Cat.'], # - №16
    ['Вы страдали ожирением?','Cat.'], # - №17
]


In [134]:
[a[0] for a in column_desc]

['Номер ответа',
 'В школе (за последние 12 месяцев)',
 'Вне школы (за последние 12 месяцев)',
 'Кибербуллинг (последние 12 месяцев?)',
 'Возраст респондента',
 'Пол респондента',
 'Количество физических атак',
 'Количество драк',
 'Ощущение одинокости',
 'Количество близких друзей',
 'Количество дней пропусков школы без разрешения',
 'Доброта и отзывчивость окружающих',
 'Частота проблем взаимопонимания с родителями',
 'Ощущение одинокости большую части (частота)',
 'Пропускали ли вы школу без разрешения (да/нет)',
 'Масса вашего тела в школе была ниже нормы?',
 'Масса вашего тела в школе была выше нормы?',
 'Вы страдали ожирением?']

In [135]:
[a[1] for a in column_desc]

['-',
 'Cat.',
 'Cat.',
 'Cat.',
 'Num.',
 'Cat.',
 'Num.',
 'Num.',
 'Cat.',
 'Num.',
 'Num',
 'Cat.',
 'Cat.',
 'Cat.',
 'Cat.',
 'Cat.',
 'Cat.',
 'Cat.']

In [136]:
[data[col].nunique() for col in data.columns]

[56981, 3, 3, 3, 9, 3, 9, 9, 6, 5, 6, 6, 6, 3, 3, 3, 3, 3]

In [154]:

headers=('Номер','Название','Описание','Тип','Кол-во вариаций')

table = np.arange(data.shape[1]*len(headers))
table = table.reshape(len(headers),data.shape[1])

table = np.array([[x for x in range(1,19)],
            data.columns,
            [a[0] for a in column_desc],
            [a[1] for a in column_desc],
            [data[col].nunique() for col in data.columns]])

table = table.transpose()

print(tabulate(table,headers=headers,tablefmt='github'))

|   Номер | Название                                         | Описание                                       | Тип   |   Кол-во вариаций |
|---------|--------------------------------------------------|------------------------------------------------|-------|-------------------|
|       1 | record                                           | Номер ответа                                   | -     |             56981 |
|       2 | Bullied_on_school_property_in_past_12_months     | В школе (за последние 12 месяцев)              | Cat.  |                 3 |
|       3 | Bullied_not_on_school_property_in_past_12_months | Вне школы (за последние 12 месяцев)            | Cat.  |                 3 |
|       4 | Cyber_bullied_in_past_12_months                  | Кибербуллинг (последние 12 месяцев?)           | Cat.  |                 3 |
|       5 | Custom_Age                                       | Возраст респондента                            | Num.  |                 9 |
|       6 | Sex     

In [138]:

print(
    tabulate(
    
            range(1,19),
            data.columns,
            [a[0] for a in column_desc],
            [a[1] for a in column_desc],
            [data[col].nunique() for col in data.columns],
        headers=('Номер','Название','Описание','Тип','Кол-во вариаций')
    )
)

TypeError: tabulate() got multiple values for argument 'headers'